# E-commerce Recommendation System

## Business Understanding

### Project Overview
E-commerce platforms rely on personalized recommendations to improve user experience, increase engagement, and drive sales. Customers interact with products through various events such as **views, clicks, and add-to-cart actions**, but these interactions are often unstructured. The goal of this project is to **build a recommendation system that predicts item properties for "add to cart" events based on prior "view" events** while also detecting abnormal user behavior to enhance recommendation accuracy.

### Business Objectives
1. **Improve Personalization**
   - Predict which product properties influence a user's decision to add an item to the cart.
   - Help businesses tailor their recommendations based on implicit browsing behavior.

2. **Reduce Noise and Improve Efficiency**
   - Identify and remove **abnormal users** who introduce bias and noise into the dataset.
   - Ensure data quality for better recommendation performance.

3. **Enhance Customer Engagement and Sales**
   - Deliver relevant product recommendations, increasing conversion rates.
   - Improve user retention by optimizing the browsing experience.

### Problem Statement
- Customers interact with multiple products before making a purchase decision. However, the properties influencing these decisions (e.g., price, brand, availability) are not explicitly logged.
- **How can we infer product properties that contribute to an "add to cart" decision based on past "view" events?**
- Additionally, **how can we detect and filter out abnormal users who distort recommendation accuracy?**

### Data Understanding
The project relies on three key datasets:

#### 1. `events.csv` (User Interaction Data)
| Column       | Description |
|-------------|------------|
| `timestamp`  | Time when the interaction occurred. |
| `visitorid`  | Unique identifier for each user. |
| `event`      | Type of interaction (e.g., view, add to cart). |
| `itemid`     | Unique identifier for each product. |
| `transactionid` | Identifies transactions (for purchases). |

#### 2. `item_properties.csv` (Product Metadata)
| Column       | Description |
|-------------|------------|
| `timestamp`  | Time when the property was recorded. |
| `itemid`     | Product identifier. |
| `property`   | Feature of the product (e.g., category, availability). |
| `value`      | Corresponding value of the property. |

#### 3. `category_tree.csv` (Product Category Data)
| Column       | Description |
|-------------|------------|
| `categoryid`  | Child category identifier |
| `parentid`     | Parent category identifier. |

### Project Scope
- **Task 1:** Develop an algorithm to predict item properties for "add to cart" events based on "view" events.
- **Task 2:** Detect abnormal users who generate noise and remove them to improve recommendation accuracy.

By addressing these tasks, the project will deliver a robust recommendation system that enhances **e-commerce personalization** and **business intelligence insights** while ensuring **clean and reliable data**. 🚀

### Hypothesis Testing
- **Null Hypothesis(Ho):**
- **Alternate Hypothesis(Ha):**

### Analytical Questions
1. How do we identify product properties that influence an "add to cart" decision?
2. What factors contribute to the decision to add an item to the cart?
3. How can we detect and filter out abnormal users who distort recommendation accuracy?
4. What are the potential impacts of this recommendation system on the e-commerce business?
5. How can we measure the success of this recommendation system and improve it over time?
6. Does visit time have influence on an "add to cart" decision?
7. What is the view to "add to cart" conversion rate?



#### Import all necessary libraries

In [ ]:
!pip install pyarrow

In [2]:
# Data Manipulation Libraries
import pandas as pd
import numpy as np
import dask.dataframe as dd

# Data Visualization Libraries
import matplotlib.pyplot as plt
import plotly.express as px

# Statistical Libraries
from scipy import stats

# Feature Engineering Libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Machine Learning Libraries
# from lightgbm import LGBMClassifier

# Metrics Libraries
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#Utils
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

print("Successfully imported all libraries...")

Successfully imported all libraries...


#### Data Understanding

In [3]:
# Load datasets
events_df = dd.read_csv('/content/drive/MyDrive/data/events.csv')
item_properties1_df = dd.read_csv('/content/drive/MyDrive/data/item_properties_part1.1.csv')
item_properties2_df = dd.read_csv('/content/drive/MyDrive/data/item_properties_part2.csv', dtype={"property":"object"})
category_tree_df = dd.read_csv('/content/drive/MyDrive/data/category_tree.csv')


In [4]:
# Load the events_df
events_df.compute()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN
...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN
2756097,1438399813142,762376,view,115946,NaN
2756098,1438397820527,1251746,view,78144,NaN
2756099,1438398530703,1184451,view,283392,NaN


In [5]:
# Load the category_tree_df
category_tree_df.compute()

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0
...,...,...
1664,49,1125.0
1665,1112,630.0
1666,1336,745.0
1667,689,207.0


In [6]:
# Load the item_properties1_df
item_properties1_df.compute()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513
...,...,...,...,...
1569806,1439694000000,86599,categoryid,618
1569807,1435460400000,153032,1066,n1020.000 424566
1569808,1440298800000,421788,888,35975 856003 37346
1569809,1437879600000,159792,400,n552.000 639502 n720.000 424566


In [7]:
# Load the item_properties2_df
item_properties2_df.compute()

,timestamp,itemid,property,value
0,1433041200000,183478,561,769062
1,1439694000000,132256,976,n26.400 1135780
2,1435460400000,420307,921,1149317 1257525
3,1431831600000,403324,917,1204143
4,1435460400000,230701,521,769062
...,...,...,...,...
1545977,1433646000000,236931,929,n12.000
1545978,1440903600000,455746,6,150169 639134
1545979,1439694000000,347565,686,610834
1545980,1433646000000,287231,867,769062


#### Concatenate the item properties dataframes

In [8]:
item_properties_df = dd.concat([item_properties1_df,item_properties2_df],ignore_index=True)

# Display the first few rows of the concatenated dataframe

item_properties_df.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [10]:
# check the shape of the new items_properties_df

item_properties_df.shape[0].compute(), item_properties_df.shape[1]

(20275902, 4)

#### Create New Features from Property column of the item_properties

In [11]:
# initialize the categoryid and available columns
item_properties_df["categoryid"] = np.nan
item_properties_df["available"] = np.nan

# Assign values to the new column based on the 'property' column
item_properties_df.assign(
    categoryid = item_properties_df["value"].where(item_properties_df["property"] == "categoryid"),
    available = item_properties_df["value"].where(item_properties_df["property"] == "available")
)

,timestamp,itemid,property,value,categoryid,available
npartitions=13,,,,,,
,int64,int64,string,string,string,string
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [12]:

# Move all the already existing numerical category ids into categoryid column
item_properties_df = item_properties_df.assign(
    categoryid = item_properties_df["property"].where(item_properties_df["property"].str.isnumeric())
)

In [13]:
# create new column called description column
item_properties_df = item_properties_df.assign(
    description=item_properties_df["value"].where(
        ~item_properties_df["property"].isin(["categoryid", "available"])
    )
)


In [14]:
# drop unnecessary property and value columns
item_properties_df = item_properties_df.drop(["property", "value"], axis=1)

In [15]:
# confirm changes
item_properties_df.compute()

,timestamp,itemid,categoryid,available,description
0,1435460400000,460429,<NA>,NaN,<NA>
1,1441508400000,206783,888,NaN,1116713 960601 n277.200
2,1439089200000,395014,400,NaN,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,NaN,n15360.000
4,1431831600000,156781,917,NaN,828513
...,...,...,...,...,...
1545977,1433646000000,236931,929,NaN,n12.000
1545978,1440903600000,455746,6,NaN,150169 639134
1545979,1439694000000,347565,686,NaN,610834
1545980,1433646000000,287231,867,NaN,769062


In [16]:
# Get info about the item_properties_df

item_properties_df.info()

<class 'dask_expr.DataFrame'>
Columns: 5 entries, timestamp to description
dtypes: float64(1), int64(2), string(2)

In [17]:
# check for null values
item_properties_df.isna().sum().compute()

,0
timestamp,0
itemid,0
categoryid,2291853
available,20275902
description,2291853


In [18]:
duplicate_counts = (
    item_properties_df.map_partitions(lambda df: df.duplicated()).sum().compute()
)
print("Total duplicate rows:", duplicate_counts)


Total duplicate rows: 19376


In [19]:
# Drop duplicates
item_properties_df = item_properties_df.drop_duplicates()

In [20]:
# confirm changes
duplicate_counts = (
    item_properties_df.map_partitions(lambda df: df.duplicated()).sum().compute()
)
print("Total duplicate rows:", duplicate_counts)


Total duplicate rows: 0


In [21]:
# Look at the events_df
events_df.info()

<class 'dask_expr.DataFrame'>
Columns: 5 entries, timestamp to transactionid
dtypes: float64(1), int64(3), string(1)

In [22]:
# check for null values
events_df.isna().sum().compute()

,0
timestamp,0
visitorid,0
event,0
itemid,0
transactionid,2733644


#### Merging Datasets
- I will first convert the timestamp to datetime so I will base merge on it
- The merge strategy to be used for merging the events_df and the item_properties is the left join taking the events_df as the main dataframe.
- I shall consider the datetime and itemid columns as the unique columns to base my joins on.
- I will finally merge the item_event dataframe to category_tree based on the categoryid to form the final_df

In [23]:
# Convert timestamp in item properties to datetime
item_properties_df["timestamp"] = dd.to_datetime(item_properties_df["timestamp"])

# Convert timestamp in events to
events_df["timestamp"] = dd.to_datetime(events_df["timestamp"])


item_properties_df["timestamp"] = item_properties_df["timestamp"].dt.floor("s")
events_df["timestamp"] = events_df["timestamp"].dt.floor("s")


In [24]:
# confirm changes
item_properties_df["timestamp"].head()

,timestamp
13,1970-01-01 00:23:59
17,1970-01-01 00:23:56
41,1970-01-01 00:23:51
79,1970-01-01 00:23:53
84,1970-01-01 00:23:53


In [25]:
# Rename timestamp columns to datetime column
item_properties_df = item_properties_df.rename(columns = {"timestamp":"datetime"})
events_df = events_df.rename(columns={"timestamp": "datetime"})

In [26]:
# Convert datetime dtype to s
item_properties_df["datetime"] = item_properties_df["datetime"].astype("datetime64[s]")
events_df["datetime"] = events_df["datetime"].astype("datetime64[s]")


In [ ]:
item_properties_df["itemid"] = item_properties_df["itemid"].astype(str)
events_df["itemid"] = events_df["itemid"].astype(str)

item_properties_df["datetime"] = item_properties_df["datetime"].astype(str)
events_df["datetime"] = events_df["datetime"].astype(str)


In [35]:

# Merge using Dask (performs parallel processing)
event_item_df = events_df.merge(item_properties_df, on=["datetime","itemid"], how="inner", indicator=True)

In [36]:
print("Unique itemid counts:")
print("item_properties_df:", item_properties_df["itemid"].nunique().compute())
print("events_df:", events_df["itemid"].nunique().compute())


Unique itemid counts:
item_properties_df: 417053
events_df: 235061


In [37]:
# count the indicator types
event_item_df["_merge"].value_counts().compute()

,count
_merge,
both,14416118
left_only,0
right_only,0


I plan to drop all the rows with _merge values of left_only and only work with the ones with both since it contains values which were merged from the two dataframes

In [38]:
# view first five rows
event_item_df.dtypes

,0
datetime,datetime64[s]
visitorid,float64
event,string[pyarrow]
itemid,float64
transactionid,float64
categoryid,string[pyarrow]
available,float64
description,string[pyarrow]
_merge,category


In [41]:
dask_df = event_item_df.compute()
pandas_df_list = []
for partition in dask_df.to_delayed():
    pandas_df_list.append(partition.compute())

# Concatenate the smaller pandas DataFrames
pandas_df = pd.concat(pandas_df_list, ignore_index=True)


KeyError: "None of [Index([None], dtype='object')] are in the [columns]"

In [ ]:
# Convert the categoryid and available to int datatype

item_event_df["categoryid"] = item_event_df["categoryid"].astype("Int64")
item_event_df["available"] = item_event_df["available"].astype("Int64")


In [ ]:
# Merge item_event_df to category_id
final_df = item_event_df.merge(category_tree_df, how="left", on="categoryid")


In [ ]:
# check the shape of the final_df
final_df.shape

(20275902, 10)

In [ ]:
# View first five rows
final_df.head()

,timestamp,itemid,categoryid,available,description,visitorid,event,transactionid,_merge,parentid
0,1435460400000,460429,1338,<NA>,NaN,NaN,NaN,NaN,left_only,1278.0
1,1441508400000,206783,888,<NA>,1116713 960601 n277.200,NaN,NaN,NaN,left_only,866.0
2,1439089200000,395014,400,<NA>,n552.000 639502 n720.000 424566,NaN,NaN,NaN,left_only,110.0
3,1431226800000,59481,790,<NA>,n15360.000,NaN,NaN,NaN,left_only,1492.0
4,1431831600000,156781,917,<NA>,828513,NaN,NaN,NaN,left_only,1374.0


In [ ]:
# check the info about final_df
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20275902 entries, 0 to 20275901
Data columns (total 10 columns):
 #   Column         Dtype   
---  ------         -----   
 0   timestamp      int64   
 1   itemid         int64   
 2   categoryid     Int64   
 3   available      Int64   
 4   description    object  
 5   visitorid      float64 
 6   event          object  
 7   transactionid  float64 
 8   _merge         category
 9   parentid       float64 
dtypes: Int64(2), category(1), float64(3), int64(2), object(2)
memory usage: 1.4+ GB


In [ ]:
item_properties_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20275902 entries, 0 to 20275901
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   timestamp    int64 
 1   itemid       int64 
 2   categoryid   object
 3   available    object
 4   description  object
dtypes: int64(2), object(3)
memory usage: 773.5+ MB


In [ ]:
# perform summary descriptive analysis on the final_df
final_df.describe().T

,count,mean,std,min,25%,50%,75%,max
timestamp,20275902.0,1435156943682.883301,3327797780.708256,1431226800000.0,1432436400000.0,1433646000000.0,1437879600000.0,1442113200000.0
itemid,20275902.0,233390.432499,134845.230654,0.0,116516.0,233483.0,350304.0,466866.0
categoryid,18772263.0,617.875911,322.80467,0.0,348.0,747.0,888.0,1697.0
available,1503639.0,0.426002,0.494494,0.0,0.0,0.0,1.0,1.0
visitorid,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
transactionid,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
parentid,18604575.0,895.854688,461.922945,8.0,602.0,866.0,1370.0,1698.0


### Exploratory Data Analysis

#### Answering Analytical Questions

#### Hypothesis Testing

## Data Preparation

## Modeling and Evaluation

## Deployment

## Conclusion